### 一、**On-Policy Monte Carlo Control**

#### **1. On and Off Policy Learning**

*   **核心问题：** 如何在与环境交互的同时，学习一个最优策略？
*   **两种范式：**
    *   **On-Policy（同策略）：** 我们要评估和改进的策略 **`π`**，与**实际生成行为轨迹（样本）的策略是同一个**。
        *   **口号：** “我正在做的，就是我正在学的。”
        *   **特点：** 通常更简单、更直接。策略通常是“软性”的，即对所有状态下的所有动作都有非零的概率被选中，以保证持续的探索。
    *   **Off-Policy（异策略）：** 我们要评估和改进的策略 **`π`**（**目标策略**），与**实际生成行为轨迹的策略 **`μ`**（**行为策略**）** 是不同的。
        *   **口号：** “我看着别人怎么做，来学习我应该怎么做。”
        *   **特点：** 更强大、更通用。可以复用由任何策略产生的历史数据，也可以学习一个确定性的最优策略，同时用一个探索性的行为策略来收集数据。

**我们现在聚焦于 On-Policy 方法。**

---

#### **2. Generalised Policy Iteration (GPI)**

这是所有强化学习控制算法的宏观蓝图。

*   **核心思想：** 在两个过程之间交替进行，直至收敛到最优策略 `π*` 和最优价值函数 `q*`。
    1.  **策略评估：** 评估当前策略 `π` 有多好。（计算 `Q^π(s, a)`）
    2.  **策略改进：** 基于评估出的价值函数，让策略变得更好。（提升 `π`）

*   **可视化理解：**
    
    策略评估和策略改进相互竞争、相互推动。评估使价值函数与当前策略一致，而改进使策略在当前价值函数下变得贪婪。当策略不再改变时，它就稳定在了最优策略上。

---

#### **3. Generalised Policy Iteration with Action-Value Function**

为什么在模型未知（无模型）的情况下，我们必须使用**动作价值函数 Q(s, a)** 而不是状态价值函数 V(s)？

*   **关键原因：** 有了 `Q(s, a)`，策略改进可以在**不依赖环境模型** `p(s‘, r | s, a)` 的情况下进行。
*   **策略改进定理：** 对于任意状态 `s`，构造一个新策略 `π'`，使得：
    `π’(s) = argmax_{a} Q^π(s, a)`
    那么，对于所有状态 `s`，新策略 `π‘` 一定不差于旧策略 `π`，即 `V^π’(s) ≥ V^π(s)`。
*   **结论：** 只要我们能够估计出 `Q^π(s, a)`，我们就可以通过简单地“在每个状态选择Q值最大的动作”来改进策略。这就是GPI在无模型条件下的实现形式。

---

#### **4. $\epsilon$-Greedy Policy Improvement**

如果我们简单地在每个状态都采取贪婪动作（`π'(s) = argmax_a Q(s, a)`），会有什么问题？

*   **问题：** 会彻底失去探索。一旦某个非最优动作在某个状态下没有被选中，它的Q值可能永远得不到更新，我们也就永远无法知道它是不是实际上更好的选择。
*   **解决方案：** **$\epsilon$-Greedy 策略。**
    *   **定义：** 以很高的概率 `1 - $\epsilon$` 选择当前认为最好的动作（贪婪动作），以很小的概率 `$\epsilon$` **随机**从所有动作中选择一个。
    *   **数学表达：**
        `π(a | s) = { 1 - $\epsilon$ + $\epsilon$ / |A(s)|, if a = argmax_{a'} Q(s, a’) }`
        `π(a | s) = { $\epsilon$ / |A(s)|, otherwise }`
    *   **优点：**
        1.  它保证了持续的探索，因为所有动作都有被选中的可能。
        2.  它同样满足**策略改进定理**。可以证明，通过 $\epsilon$-greedy 策略改进得到的新策略，其价值函数 `V^{π'}(s)` 确实不低于旧策略 `V^π(s)`。这意味着我们仍然在朝着最优策略的方向前进。

---

#### **5. Monte Carlo Policy Improvement**

现在，我们将GPI和蒙特卡洛方法结合起来。

*   **目标：** 在不知道环境模型的情况下，通过与环境交互的完整经验轨迹来学习最优策略。
*   **方法：** 使用蒙特卡洛方法来估计动作价值函数 `Q(s, a)`。
*   **基本流程（On-Policy MC Control）：**
    1.  **初始化：** 随机初始化一个策略 `π` 和 Q表 `Q(s, a)`。
    2.  **循环（多个Episode）：**
        *   **策略评估（蒙特卡洛）：** 使用策略 `π` 生成一个完整的Episode（例如：`S0, A0, R1, S1, A1, R2, ..., ST`）。对于这个Episode中出现的每一个状态-动作对 `(s, a)`，计算其从该时刻起获得的实际回报 `G_t`。然后用这个 `G_t` 去更新 `Q(s, a)` 的估计（例如：增量平均 `Q(s, a) ← Q(s, a) + α (G_t - Q(s, a))`）。
        *   **策略改进：** 对于这个Episode中访问到的每一个状态 `s`，根据更新后的 `Q(s, ·)` 来改进策略 `π`，例如将其更新为关于 `Q(s, ·)` 的 $\epsilon$-greedy 策略。

---

#### **6. GLIE Monte Carlo Control**

基本的On-Policy MC Control能工作，但理论上不能保证收敛到最优策略 `π*`。为了确保收敛，我们需要满足 **GLIE** 条件。

*   **GLIE 定义：** Greedy in the Limit with Infinite Exploration（无限探索下的极限贪婪）
    1.  **无限探索：** 所有的状态-动作对 `(s, a)` 都会被访问无限次。
        `lim (N(s, a) → ∞)` for all s, a
    2.  **极限贪婪：** 策略最终会收敛到一个贪婪策略。
        `lim (π(a | s) → 1)` for `a = argmax_{a'} Q(s, a’)`, and to 0 otherwise.

*   **如何实现GLIE？**
    *   **GLIE MC Control 算法：**
        1.  使用 $\epsilon$-greedy 策略来采集Episode。
        2.  对于Episode中每个出现的状态-动作对 `(s, a)`：
            *   更新访问计数：`N(s, a) ← N(s, a) + 1`
            *   更新Q值：`Q(s, a) ← Q(s, a) + (1 / N(s, a)) * (G_t - Q(s, a))` （这里使用简单的平均，也可以用学习率 `α`）
        3.  **关键步骤：** 在策略改进阶段，**让探索概率 $\epsilon$ 逐渐衰减到0**。
            *   例如：`$\epsilon$ ← 1 / k`，其中 `k` 是Episode的计数。

*   **为什么GLIE有效？**
    *   在开始时，`$\epsilon$` 很大，保证了充分的探索，满足条件1。
    *   随着时间推移，`$\epsilon$` 逐渐减小到0，策略变得越来越贪婪，最终收敛到一个确定性的最优策略，满足条件2。
    *   在Q函数的估计上，因为每个 `(s, a)` 对被访问了无限次，根据大数定律，`Q(s, a)` 会收敛到真实的 `q_π(s, a)`。

### **总结**

你的提纲完美地展现了On-Policy MC Control的推导过程：

1.  **确立范式（On-Policy）** -> 
2.  **确立宏观框架（GPI）** -> 
3.  **确定实现GPI的工具（Action-Value Function Q）** -> 
4.  **解决贪婪探索矛盾（$\epsilon$-Greedy）** -> 
5.  **组合成基础算法（MC Policy Improvement）** -> 
6.  **为算法加上理论保证（GLIE MC Control）**



### 二、On Policy TD Learning

### 1. Sarsa Algorithm

**核心思想**：在同策略（on-policy）下，使用时序差分方法直接学习动作价值函数 $Q(s, a)$。其更新基于五元组$(S_t, A_t, R_{t+1}, S_{t+1}, A_{t+1})$。

- **更新公式**：
  $Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha [R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)]$
  其中$A_{t+1}$ 由当前策略（如ε-greedy）在$S_{t+1}$ 状态产生。

- **算法流程**：
  1.  初始化$Q(s, a)$。
  2.  对每个episode：
      a. 初始化状态$S$，并根据Q值选择动作$A$（如ε-greedy）。
      b. 每一步：
          i. 执行动作$A$，观测$R, S'$。
          ii. 在$S‘$ 根据当前策略选择$A’$。
          iii. 使用上述公式更新$Q(S, A)$。
          iv.$S \leftarrow S‘$,$A \leftarrow A’$。
      c. 直到$S$ 为终止状态。

---

### 2. n-step Sarsa

**核心思想**：将Sarsa的单步回报扩展为n步回报，以平衡TD的偏差和MC的方差。

- **n-step Return**：
  $G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \dots + \gamma^{n-1} R_{t+n} + \gamma^n Q(S_{t+n}, A_{t+n})$

- **更新公式**：
  $Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha [G_{t:t+n} - Q(S_t, A_t)]$

- **特点**：n=1时退化为Sarsa，n很大时接近蒙特卡洛方法。

---

### 3. Sarsa(λ)

**核心思想**：不局限于单一n步，而是通过λ参数**平滑地平均所有可能的n-step Sarsa回报**，从而更高效地进行信用分配。

- **λ-return**：
  $G_t^\lambda = (1-\lambda) \sum_{n=1}^{\infty} \lambda^{n-1} G_{t:t+n}$
  （实践中为有限和）

- **目标**：通过更新Q值，使其更接近 $G_t^\lambda$。

---

### 4. Forward View and Backward View

这是理解Sarsa(λ)的两个等价视角。

#### Forward View (前向视角)

- **定义**：在时刻t，**向前看**，等待并观察到直到幕结束的所有数据后，计算λ-return$G_t^\lambda$，然后用它来更新$Q(S_t, A_t)$。
- **特点**：**理论模型**，**非在线**，必须等到幕结束才能更新。

#### Backward View (后向视角)

- **定义**：在每一步t，**向后看**，根据当前TD Error和资格迹，**同时更新所有相关的状态-动作对**的Q值。
- **TD Error**：
  $\delta_t = R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)$
- **更新规则**：对于所有$s, a$，
  $Q(s, a) \leftarrow Q(s, a) + \alpha \delta_t E_t(s, a)$
- **特点**：**在线**、**增量式**、**计算高效**。**我们实际使用的是这种实现方式。**

---

### 5. Eligibility Trace

**核心思想**：一个短期记忆向量，用于记录每个状态-动作对近期被访问的频率和时间远近，从而解决信用分配问题。

- **迹的更新公式（累积迹）**：
 $
  \begin{aligned}
  E_0(s, a) &= 0 \\
  E_t(s, a) &= \gamma \lambda E_{t-1}(s, a) + \mathbf{1}(S_t = s, A_t = a)
  \end{aligned}
 $
  其中$\mathbf{1}(\cdot)$ 是指示函数。

- **作用机制**：
  - 当一个状态-动作对被访问时，其迹**增加1**。
  - 在每一步，所有迹以 $\gamma \lambda$ 的速率**衰减**。
  - TD Error $\delta_t$ 根据迹的强度 $E_t(s, a)$ 比例性地更新**所有**状态-动作对的Q值。

- **直观理解**：迹像“脚印”，脚印越深（迹值越大）的状态-动作对，对当前结果“功劳”越大，因此获得更多的更新份额。

### 问题：**我们到底在计算什么？以及计算出的Q值是如何改变策略的？**

让我们彻底澄清这个问题。

### 我们最终在计算什么？

**答案：我们最终计算的是最优动作价值函数 Q*(s, a)。**

它的定义是：
$Q*(s, a) = 从状态s执行动作a开始，之后一直采取最优策略所能获得的期望累计回报$

**通俗解释**：$Q*(s, a)$ 这张表告诉你，在每一个状态$s$下，每一个可选动作$a$的**长期“好坏”得分**。这个分数考虑了立即奖励和所有未来的最佳回报。

**最终目标**：学完算法后，我们得到的就是这张充满分数的Q表。

---

### Q值是如何改变策略的？

策略 $π(s)$ 就是一个函数，它告诉你在状态$s$应该做什么动作。**Q值通过成为策略的“决策依据”来改变它。**

这个过程可以分为 **学习过程中** 和 **学习结束后** 两个阶段。

#### 1. 学习过程中：策略的渐进式改进

在Sarsa学习过程中，我们同时在做两件事：
1.  **评估**：更新Q值，使其更准确地反映**当前策略**的好坏。
2.  **改进**：基于更新后（更准确）的Q值，让策略变得“更贪婪”。

这是一个动态的、循环的过程。我们使用一个 $ε-greedy$ 策略来说明：

- **初始**：Q值全是随机数，策略几乎是随机的。
- **第一步（评估）**：在状态$s$，根据当前ε-greedy策略（比如90%概率选当前Q值最高的动作，10%概率随机探索）选择动作$a$。执行后，用Sarsa公式更新 $Q(s, a)$。
    $Q(s, a) = Q(s, a) + α * [ r + γ * Q(s', a') - Q(s, a) ]$
- **第二步（改进）**：现在，$Q(s, a)$ 被更新了。那么，**下一次**你再访问状态$s$时，你的ε-greedy策略所依赖的Q值已经变了！
    - 如果动作$a1$的Q值因为这次更新**变高**了，那么下次选择它的**概率就会增加**。
    - 如果动作$a2$的Q值**变低**了，那么下次选择它的**概率就会降低**。

**核心**：策略 $π$ 是 **Q值的函数**。Q值一变，策略自然就变了。你不需要显式地去“修改”策略，你只需要**更新Q值，策略会自动随之优化**。

#### 2. 学习结束后：提取最优策略

当Q值收敛到最优Q*后，我们可以从中**提取出最优策略 π***。方法极其简单：

**贪婪策略：**
$π*(s) = argmax_{a} Q*(s, a)$

**通俗解释**：在每个状态$s$，只需要**查Q表**，看看哪个动作$a$对应的Q值最大，就执行哪个动作。这个策略就是最优策略。


### 总结：Q值与策略的关系

可以把Q值想象成一个 **“策略顾问”**：

- **学习过程**：你不断向这个顾问提供新的经验（$(s,a,r,s',a')$），他不断修正自己的建议（**更新Q表**）。
- **决策过程**：在需要做决定时，你（策略）会**听取顾问的建议**（查询Q表），大部分时间采纳他的最佳推荐（greedy），偶尔也试试其他可能（exploration）。
- **最终成果**：当这个顾问的经验足够丰富（Q表收敛），他的建议就是最优的。此时，你只要完全听从他的最佳推荐（$argmax Q$），你就能做出最优决策。

所以，**Sarsa等算法的核心就是通过经验数据来训练这位“顾问”（Q函数），策略的优化是这个过程的自然结果。**

### 三、**Off-Policy Learning** 

---

### 1. Goal and Importance

**核心思想**：使智能体能够通过遵循一个行为策略（**behavior policy**）产生的数据，来学习一个不同的目标策略（**target policy**）。

- **行为策略 (μ)**：智能体**实际执行**的策略，用于与环境交互、收集数据。通常具有探索性（如ε-greedy）。
- **目标策略 (π)**：智能体**想要学习**的最终策略。通常是当前最优策略的估计（如greedy策略）。

**目标**：
- **主要目标**：学习最优策略，同时保持充分的探索。这是Q-Learning的基础。
- **其他重要目标**：
    1.  **从观察中学习**：通过观察其他智能体（甚至是人类）的行为（行为策略）来学习目标策略。
    2.  **重用旧数据**：利用由**过去策略**或**不同策略**收集到的历史经验池（Experience Replay），打破数据间的关联性，提高数据效率。
    3.  **多策略学习**：从单一行为策略产生的数据中，同时学习多个目标策略。

**重要性**：离策略学习将**“行为”与“学习目标”** 分离开，是实现高效、稳定、数据驱动强化学习的关键，也是许多先进算法（如DQN、DDPG）的核心组成部分。

---

### 2. Importance Sampling

**核心思想**：一种估计一个分布（目标策略π）下的期望值，而只有来自另一个分布（行为策略μ）的样本时，所使用的统计技术。

- **数学基础**：
 $
  \mathbb{E}_{X \sim p}[f(X)] = \mathbb{E}_{X \sim q}\left[\frac{p(X)}{q(X)} f(X)\right]
 $
  其中$\frac{p(X)}{q(X)}$称为**重要性权重**。

- **在强化学习中的应用**：用于校正行为策略和目标策略选择动作概率的差异，从而用μ的轨迹来估计π的价值。
    - 一条轨迹$A_t, S_{t+1}, A_{t+1}, ..., S_T$的重要性权重（乘积）为：
   $
    \rho_t^T = \prod_{k=t}^{T-1} \frac{\pi(A_k|S_k)}{\mu(A_k|S_k)}
   $
    - **普通重要性采样**：直接使用$\rho_t^T$进行加权平均。
    - **加权重要性采样**：使用$\rho_t^T$进行加权平均，但除以权重的总和以减小方差。

**重要性**：提供了离策略学习的理论基石，使得利用任何行为策略产生的数据来评估目标策略成为可能。但其**高方差**是一个主要挑战。

---

### 3. Q-Learning

**核心思想**：一种直接、优雅的离策略控制算法，通过行为策略探索，但直接更新对最优价值函数的估计。

- **算法核心：Q-Learning Update**：
 $
  Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_{a} Q(S_{t+1}, a) - Q(S_t, A_t) \right]
 $

- **策略角色**：
    - **行为策略 (μ)**：可以是任何能保证充分探索的策略（如ε-greedy）。它负责选择动作$A_t$和$A_{t+1}$？不，Q-Learning在更新时**不关心**$A_{t+1}$是什么。
    - **目标策略 (π)**：是完全贪婪的策略$\pi(s) = \arg\max_a Q(s, a)$。更新目标中的$\max_{a} Q(S_{t+1}, a)$正对应了这个贪婪目标策略的价值。

- **离策略性的体现**：
    更新公式的目标是$R_{t+1} + \gamma \max_{a} Q(S_{t+1}, a)$。这个目标**完全由贪婪的目标策略决定**，与行为策略实际选择的下一个动作$A_{t+1}$**无关**。智能体通过行为策略探索，但学习的是“如果我一直采取最优动作，价值会是多少”。

**重要性**：Q-Learning是强化学习中最重要、应用最广泛的算法之一。它简单、高效，且不需要重要性采样，避免了高方差问题，是许多深度强化学习算法（如DQN）的直接基础。